In [4]:
import heapq
import tkinter as tk
import time
import tkinter.simpledialog as simpledialog

class Node:
    def __init__(self, position, parent=None):
        self.position = position
        self.parent = parent
        self.g = 0  # Cost from start to node
        self.h = 0  # Heuristic cost to goal
        self.f = 0  # Total cost

    def __lt__(self, other):
        return self.f < other.f

def heuristic(a, b):
    return abs(a[0] - b[0]) + abs(a[1] - b[1])

def a_star_search(grid, start, goal):
    open_list = []
    closed_set = set()

    start_node = Node(start)
    goal_node = Node(goal)

    heapq.heappush(open_list, start_node)

    while open_list:
        current_node = heapq.heappop(open_list)

        if current_node.position == goal:
            path = []
            while current_node:
                path.append(current_node.position)
                current_node = current_node.parent
            return path[::-1]  # Return reversed path

        closed_set.add(current_node.position)

        for new_position in [(0, 1), (1, 0), (0, -1), (-1, 0)]:
            node_position = (current_node.position[0] + new_position[0], current_node.position[1] + new_position[1])

            if (0 <= node_position[0] < len(grid)) and (0 <= node_position[1] < len(grid[0])):
                if grid[node_position[0]][node_position[1]] == 1 and node_position not in closed_set:
                    child_node = Node(node_position, current_node)
                    child_node.g = current_node.g + 1
                    child_node.h = heuristic(child_node.position, goal_node.position)
                    child_node.f = child_node.g + child_node.h

                    if add_to_open(open_list, child_node):
                        heapq.heappush(open_list, child_node)

    return None  # No path found

def add_to_open(open_list, child_node):
    for node in open_list:
        if child_node.position == node.position and child_node.g > node.g:
            return False
    return True

class BotGrid(tk.Tk):
    def __init__(self, grid, bots, paths):
        super().__init__()
        self.title("Two Bots Moving on a Grid")

        self.grid = grid
        self.bots = bots
        self.paths = paths
        self.bot_positions = [path[0] for path in paths]  # Start at first position
        self.bot_squares = []

        self.rows = len(grid)
        self.cols = len(grid[0])
        self.cell_size = 100

        self.canvas = tk.Canvas(self, width=self.cols * self.cell_size, height=self.rows * self.cell_size)
        self.canvas.pack()

        self.create_grid()
        self.create_bots()

        # Timer and step counters
        self.start_time = time.time()
        self.steps = [0, 0]  # Steps for bot 0 and bot 1
        self.timer_label = tk.Label(self, text="Time: 0s")
        self.timer_label.pack()
        self.steps_label = tk.Label(self, text="Bot 0 steps: 0, Bot 1 steps: 0")
        self.steps_label.pack()

        self.move_bots()

    def create_grid(self):
        for r in range(self.rows):
            for c in range(self.cols):
                x1, y1 = c * self.cell_size, r * self.cell_size
                x2, y2 = x1 + self.cell_size, y1 + self.cell_size
                color = "white" if self.grid[r][c] == 1 else "black"
                self.canvas.create_rectangle(x1, y1, x2, y2, fill=color)

    def create_bots(self):
        for bot in range(2):
            r, c = self.bot_positions[bot]
            x1, y1 = c * self.cell_size, r * self.cell_size
            x2, y2 = x1 + self.cell_size, y1 + self.cell_size
            color = "red" if bot == 0 else "blue"
            bot_square = self.canvas.create_rectangle(x1, y1, x2, y2, fill=color)
            self.bot_squares.append(bot_square)

    def move_bots(self):
        self.i, self.j = 0, 0  # Indices for bot 0 and bot 1

        def move_step():
            next_positions = {}
            collisions = set()

            # First pass: Check next positions for collisions
            for bot in [0, 1]:
                if bot == 0 and self.i < len(self.paths[bot]):
                    next_position = self.paths[bot][self.i]
                elif bot == 1 and self.j < len(self.paths[bot]):
                    next_position = self.paths[bot][self.j]
                else:
                    continue

                if next_position in next_positions:
                    collisions.add(next_position)
                else:
                    next_positions[next_position] = bot

            # Second pass: Move bots, handle potential collisions
            for bot in [0, 1]:
                if bot == 0 and self.i < len(self.paths[bot]):
                    current_position = self.paths[bot][self.i]
                elif bot == 1 and self.j < len(self.paths[bot]):
                    current_position = self.paths[bot][self.j]
                else:
                    continue

                if current_position in collisions:
                    other_bot = next_positions[current_position]
                    if bot == 0:
                        self.move_bot(bot, current_position)
                        self.steps[bot] += 1
                        self.i += 1
                    else:
                        continue
                else:
                    self.move_bot(bot, current_position)
                    self.steps[bot] += 1
                    if bot == 0:
                        self.i += 1
                    else:
                        self.j += 1

            # Update timer and steps
            elapsed_time = time.time() - self.start_time
            self.timer_label.config(text=f"Time: {int(elapsed_time)}s")
            self.steps_label.config(text=f"Bot 0 steps: {self.steps[0] -1}, Bot 1 steps: {self.steps[1] -1}")

            if self.i < len(self.paths[0]) or self.j < len(self.paths[1]):
                self.after(500, move_step)  # Call after 500ms for smooth movement

        move_step()

    def move_bot(self, bot, next_position):
        r, c = next_position
        x1, y1 = c * self.cell_size, r * self.cell_size
        x2, y2 = x1 + self.cell_size, y1 + self.cell_size
        self.canvas.coords(self.bot_squares[bot], x1, y1, x2, y2)

# User input for grid size, obstacles, and bots
rows = int(simpledialog.askstring("Grid Size", "Enter number of rows:"))
cols = int(simpledialog.askstring("Grid Size", "Enter number of columns:"))

# Initialize the grid
grid = [[1 for _ in range(cols)] for _ in range(rows)]

# Get obstacles from the user
obstacle_input = simpledialog.askstring("Obstacles Input", "Enter obstacles in the form 'row,col' (e.g., '1,1;2,3'):").strip()
if obstacle_input:
    for obs in obstacle_input.split(';'):
        r, c = map(int, obs.split(','))
        grid[r][c] = 0  # Mark the position as an obstacle

# Get bot configurations from the user
num_bots = int(simpledialog.askstring("Bots Input", "Enter number of bots:"))
bots = []
for i in range(num_bots):
    start = tuple(map(int, simpledialog.askstring("Bot Input", f"Enter start position for Bot {i + 1} (row,col):").split(',')))
    goal = tuple(map(int, simpledialog.askstring("Bot Input", f"Enter goal position for Bot {i + 1} (row,col):").split(',')))
    bots.append({'start': start, 'goal': goal})

# Generate paths for bots using A* search
paths = []
for bot in bots:
    path = a_star_search(grid, bot['start'], bot['goal'])
    paths.append(path)

# print(paths)
# Start the Tkinter simulation
app = BotGrid(grid, bots, paths)
app.mainloop()


[[(0, 0), (0, 1), (0, 2), (1, 2), (2, 2), (2, 3)], [(0, 4), (0, 3), (0, 2), (1, 2), (2, 2), (2, 1)]]
